In [20]:
#import libiraries
from bs4 import BeautifulSoup as bs
import requests
from word2number import w2n
import sqlite3
import pandas as pd


In [21]:
# url related to each book 
related_category_url= "https://books.toscrape.com/catalogue/"

In [23]:
#create data base
connection=sqlite3.connect("books.db")
cursor=connection.cursor()
#create two tables 
cursor.execute('''CREATE TABLE IF NOT EXISTS books(id Text,title Text,price Float,rates text ,PRIMARY KEY (id))''')
cursor.execute('''CREATE TABLE IF NOT EXISTS catigories (id Text, category Text ,FOREIGN KEY(id) REFERENCES books(id)
ON UPDATE CASCADE
ON DELETE CASCADE)''')

In [24]:
#scraping all wepsite pages and get info from books
page=1
books_id=0
index1=1
index2=1
index3=1
ra=[]
while page != 51:
    url=f"https://books.toscrape.com/catalogue/category/books_1/page-{page}.html"
    respons=requests.get(url)
    soup=bs(respons.text,"html.parser")
    #get snaps from soup to get all data befor iterarion
    prices=soup.find_all('div',attrs={'class','product_price'})
    rates=soup.find_all('p')
    categories=soup.find_all('h3')    
    titels=soup.find_all('h3')
    #get category
    for c in categories:
        book_url= (c).find('a').get('href')   #get the url for each book
        s=book_url.replace("../../","")
        res=requests.get(related_category_url+s)    
        cat_soup=bs(res.text,'html.parser')
        cat=cat_soup.find('ul').find_all('li')[2].find('a').get_text()
        books_id=books_id+1
        cursor.execute("INSERT INTO books(id) VALUES(?);",[books_id])              
        cursor.execute("INSERT INTO catigories(id,category) VALUES(?,?);",[books_id,cat])  #insert value into the coulmn
    #get title
    for t in titels:
        A= (t).find('a').get('title')
        cursor.execute('''UPDATE books SET title=? WHERE id=? ''',[A,index1])
        index1=index1+1
     #get price     
    for b in prices:
        price=(b).find('p')
        P=float((price.text).replace('Â£',''))
        cursor.execute('''UPDATE books SET price =? WHERE id=?''',[P,index2])
        index2=index2+1
    #get rate
    for r in rates:
        rate=(r).get(('class'))
        if rate[0]=='star-rating':
            R=int(w2n.word_to_num(rate[1]))
            ra.append(R)
            cursor.execute('''UPDATE books SET rates=? WHERE id=?''',[R,index3])
            index3=index3+1
    connection.commit()
    page=page+1         #increse the page to get the next page
#close connection with db               
connection.close()              

In [25]:
connection=sqlite3.connect("books.db")
df = pd.read_sql('''SELECT *  
                    FROM catigories;''',connection)
df


,id,category
0,1,Poetry
1,2,Historical Fiction
2,3,Fiction
3,4,Mystery
4,5,History
...,...,...
995,996,Classics
996,997,Sequential Art
997,998,Historical Fiction
998,999,Mystery


In [26]:
connection=sqlite3.connect("books.db")
df = pd.read_sql('''SELECT *  
                    FROM books;''',connection)
df


,id,title,price,rates
0,1,A Light in the Attic,51.77,3
1,2,Tipping the Velvet,53.74,1
2,3,Soumission,50.10,1
3,4,Sharp Objects,47.82,4
4,5,Sapiens: A Brief History of Humankind,54.23,5
...,...,...,...,...
995,996,Alice in Wonderland (Alice's Adventures in Won...,55.53,1
996,997,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4
997,998,A Spy's Devotion (The Regency Spies of London #1),16.97,5
998,999,1st to Die (Women's Murder Club #1),53.98,1
